# Contract Processing Agent - Test Suite

Comprehensive test suite for the Contract Rule Extractor Agent.

## Test Coverage:
- **Confidence Scoring** - Keyword matching and text quality validation
- **Rule Extraction** - Rule structure and mapping verification
- **Document Parsing** - File format support and error handling
- **RAG Questions** - Question specificity and coverage
- **Rule Refinement** - Priority assignment and description validation
- **Integration** - End-to-end workflow testing

**Author:** r4 Technologies, Inc 2025  
**Version:** 1.0

In [ ]:
import unittest
import json
import tempfile
from pathlib import Path
from unittest.mock import Mock, patch, MagicMock

print("[OK] Test dependencies imported successfully")

## Test Classes

The test suite is organized into the following test classes:

### TestConfidenceScoring

In [ ]:
class TestConfidenceScoring(unittest.TestCase):
    """Test confidence scoring based on keyword matching."""
    
    def setUp(self):
        """Initialize test fixtures."""
        self.agent = Mock()
        self.agent._calculate_confidence = self._mock_calculate_confidence
        
        self.confidence_keywords = {
            "payment_terms": ["net", "days", "payment", "terms", "due date", "invoice date"],
            "approval_process": ["approve", "approval", "authorized", "manager", "director"],
            "late_penalties": ["penalty", "late", "overdue", "interest", "per month"],
        }
    
    def _mock_calculate_confidence(self, text, rule_type):
        """Mock confidence calculation."""
        if not text or "not specified" in text.lower():
            return "low"
        
        keywords = self.confidence_keywords.get(rule_type, [])
        text_lower = text.lower()
        keyword_matches = sum(1 for keyword in keywords if keyword.lower() in text_lower)
        keyword_ratio = keyword_matches / max(len(keywords), 1)
        text_length = len(text.split())
        
        if keyword_ratio >= 0.6 and text_length >= 15:
            return "high"
        elif keyword_ratio >= 0.3 and text_length >= 10:
            return "medium"
        else:
            return "low"
    
    def test_high_confidence_scoring(self):
        """Test high confidence scoring with multiple keywords."""
        text = "Payment terms are Net 30 days from invoice date. Payment must be made within the specified terms."
        confidence = self.agent._calculate_confidence(text, "payment_terms")
        self.assertEqual(confidence, "high")
    
    def test_medium_confidence_scoring(self):
        """Test medium confidence scoring with some keywords."""
        text = "Payment is due within 30 days."
        confidence = self.agent._calculate_confidence(text, "payment_terms")
        self.assertEqual(confidence, "medium")
    
    def test_low_confidence_scoring(self):
        """Test low confidence scoring with few keywords."""
        text = "Invoice processing."
        confidence = self.agent._calculate_confidence(text, "payment_terms")
        self.assertEqual(confidence, "low")
    
    def test_not_specified_returns_low(self):
        """Test that 'Not specified' returns low confidence."""
        text = "Not specified"
        confidence = self.agent._calculate_confidence(text, "payment_terms")
        self.assertEqual(confidence, "low")




### TestRuleExtraction

In [ ]:
class TestRuleExtraction(unittest.TestCase):
    """Test rule extraction from contract text."""
    
    def test_rule_mapping_completeness(self):
        """Test that all rule types are properly mapped."""
        rule_mapping = {
            "payment_terms": {"type": "payment_term", "priority": "high"},
            "approval_process": {"type": "approval", "priority": "high"},
            "late_penalties": {"type": "penalty", "priority": "high"},
            "submission_requirements": {"type": "submission", "priority": "high"},
            "discount_terms": {"type": "discount", "priority": "medium"},
            "dispute_resolution": {"type": "dispute_resolution", "priority": "medium"},
            "tax_requirements": {"type": "tax", "priority": "high"},
            "currency_requirements": {"type": "currency", "priority": "medium"},
            "invoice_validity": {"type": "validity", "priority": "medium"},
            "payment_methods": {"type": "payment_method", "priority": "medium"},
        }
        
        # Verify all rule types have both type and priority
        for key, mapping in rule_mapping.items():
            self.assertIn("type", mapping)
            self.assertIn("priority", mapping)
            self.assertIn(mapping["priority"], ["high", "medium", "low"])
    
    def test_rule_structure(self):
        """Test that extracted rules have correct structure."""
        rule = {
            "rule_id": "payment_terms",
            "type": "payment_term",
            "description": "Net 30 days from invoice date",
            "priority": "high",
            "confidence": "high",
        }
        
        # Verify all required fields
        required_fields = ["rule_id", "type", "description", "priority", "confidence"]
        for field in required_fields:
            self.assertIn(field, rule)
        
        # Verify confidence is valid
        self.assertIn(rule["confidence"], ["high", "medium", "low"])
    
    def test_rule_filtering_empty_descriptions(self):
        """Test that empty descriptions are filtered out."""
        raw_rules = {
            "payment_terms": "Net 30 days",
            "approval_process": "Not found",
            "late_penalties": "1.5% per month",
        }
        
        # Only non-"Not found" rules should be processed
        valid_rules = {k: v for k, v in raw_rules.items() if v != "Not found"}
        self.assertEqual(len(valid_rules), 2)




### TestDocumentParsing

In [ ]:
class TestDocumentParsing(unittest.TestCase):
    """Test document parsing functionality."""
    
    def test_supported_file_formats(self):
        """Test that PDF and DOCX formats are supported."""
        supported_formats = [".pdf", ".docx"]
        
        for fmt in supported_formats:
            self.assertTrue(fmt.lower() in [".pdf", ".docx"])
    
    def test_unsupported_file_format_raises_error(self):
        """Test that unsupported formats raise ValueError."""
        unsupported_format = ".txt"
        supported_formats = [".pdf", ".docx"]
        
        self.assertNotIn(unsupported_format, supported_formats)
    
    def test_file_not_found_raises_error(self):
        """Test that missing files raise FileNotFoundError."""
        file_path = Path("/nonexistent/path/file.pdf")
        self.assertFalse(file_path.exists())




### TestRAGQuestions

In [ ]:
class TestRAGQuestions(unittest.TestCase):
    """Test RAG extraction questions."""
    
    def test_all_rule_types_have_questions(self):
        """Test that all rule types have corresponding questions."""
        questions = {
            "payment_terms": "What are the payment terms?",
            "approval_process": "What is the invoice approval process?",
            "late_penalties": "What are the late payment penalties?",
            "submission_requirements": "What must be included on every invoice?",
            "discount_terms": "What early payment discounts are available?",
            "dispute_resolution": "What is the process for resolving disputes?",
            "tax_requirements": "What are the tax requirements?",
            "currency_requirements": "What currency must invoices be in?",
            "invoice_validity": "What is the validity period for invoices?",
            "payment_methods": "What payment methods are accepted?",
        }
        
        # Verify we have 10 questions
        self.assertEqual(len(questions), 10)
        
        # Verify each question is non-empty
        for key, question in questions.items():
            self.assertGreater(len(question), 0)
            self.assertIn("?", question)
    
    def test_question_specificity(self):
        """Test that questions are specific and actionable."""
        questions = {
            "payment_terms": "What are the payment terms (Net days, PO requirements, payment methods)?",
            "approval_process": "What is the invoice approval process and who must approve?",
        }
        
        for key, question in questions.items():
            # Questions should contain specific keywords
            self.assertGreater(len(question), 20)
            self.assertIn("?", question)




### TestRuleRefinement

In [ ]:
class TestRuleRefinement(unittest.TestCase):
    """Test rule refinement and structuring."""
    
    def test_rule_priority_assignment(self):
        """Test that rules are assigned correct priorities."""
        high_priority_types = ["payment_term", "approval", "penalty", "tax"]
        medium_priority_types = ["discount", "dispute_resolution", "currency", "validity", "payment_method"]
        
        for rule_type in high_priority_types:
            self.assertIn(rule_type, high_priority_types)
        
        for rule_type in medium_priority_types:
            self.assertIn(rule_type, medium_priority_types)
    
    def test_minimum_description_length(self):
        """Test that descriptions meet minimum length requirement."""
        min_length = 15
        
        descriptions = [
            "Net 30 days from invoice date",  # Valid
            "Short",  # Invalid
            "Payment terms are Net 30 days from invoice date with monthly installments",  # Valid
        ]
        
        valid_descriptions = [d for d in descriptions if len(d) > min_length]
        self.assertEqual(len(valid_descriptions), 2)




### TestIntegration

In [ ]:
class TestIntegration(unittest.TestCase):
    """Integration tests for the complete workflow."""
    
    def test_extraction_pipeline_structure(self):
        """Test the complete extraction pipeline structure."""
        pipeline_steps = [
            "parse_document",
            "_create_vectorstore",
            "extract_rules",
            "refine_rules",
            "run",
        ]
        
        # Verify all steps are defined
        for step in pipeline_steps:
            self.assertIsNotNone(step)
    
    def test_rule_output_format(self):
        """Test that rule output is properly formatted."""
        rules = [
            {
                "rule_id": "payment_terms",
                "type": "payment_term",
                "description": "Net 30 days from invoice date",
                "priority": "high",
                "confidence": "high",
            },
            {
                "rule_id": "late_penalties",
                "type": "penalty",
                "description": "1.5% per month on overdue balance",
                "priority": "high",
                "confidence": "medium",
            },
        ]
        
        # Verify output can be serialized to JSON
        json_output = json.dumps(rules, indent=2)
        self.assertIsNotNone(json_output)
        
        # Verify we can deserialize it back
        parsed_rules = json.loads(json_output)
        self.assertEqual(len(parsed_rules), 2)


if __name__ == "__main__":
    unittest.main()


## Running the Tests

Execute the cell below to run all tests:

In [ ]:
# Run all tests
if __name__ == "__main__":
    # Create test suite
    loader = unittest.TestLoader()
    suite = unittest.TestSuite()
    
    # Add all test classes
    suite.addTests(loader.loadTestsFromTestCase(TestConfidenceScoring))
    suite.addTests(loader.loadTestsFromTestCase(TestRuleExtraction))
    suite.addTests(loader.loadTestsFromTestCase(TestDocumentParsing))
    suite.addTests(loader.loadTestsFromTestCase(TestRAGQuestions))
    suite.addTests(loader.loadTestsFromTestCase(TestRuleRefinement))
    suite.addTests(loader.loadTestsFromTestCase(TestIntegration))
    
    # Run tests
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    
    # Print summary
    print("\n" + "="*60)
    print(f"Tests run: {result.testsRun}")
    print(f"Failures: {len(result.failures)}")
    print(f"Errors: {len(result.errors)}")
    print(f"Success: {result.wasSuccessful()}")
    print("="*60)